1. What are the main motivations for reducing a dataset’s dimensionality? What are the main drawbacks?

The main motivations are to:

1. Reduce the dimensionality which can greatly speed up training and prediction time.
2. Visualize the data by projecting it to 2 or 3 dimensions.
3. Make the decision surface simpler by discovering a lower-dimensional manifold that the data lies near.
4. Data compression.

The main drawbacks

1. Some information will be lost from the data in the process of dimensionality reduction. In some cases this can reduce the quality of the model.
2. It can increase the complexity of an ML pipeline.
3. It can be computationally expensive. This can offset the gains realized from training speedup.

2. What is the curse of dimensionality?

The curse of dimensionality refers the challenge posed to learning in high-dimensional data. As dimensionality increases, unintuitive things happen:

- As the number of dimensions $n \to \infty$, the vast majority of volume in a shape like a hypersphere or hypercube becomes increasingly near the surface of the shape.
- The average distance between randomly selected points increases significantly. This leads to very sparse data, and increases the risk of overfitting the data.
- In order to maintain the same density of data points as the dimensions increase, the number of points needed is exponential in the number of dimensions.

3. Once a dataset’s dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?

It is possible to approximately reconstruct the data, but the reconstruction will be lossy. This is because information is necessarily lost when reducing the dimensions.

In PCA, the principal components with the lowest explained variance are discarded, so the reconstruction can be fairly accurate if you keep for example 95% of the explained variance. The reconstructed points will lie at the closest point to the original point within the d-dimensional hyperplane formed by the d chosen principal components.

In Randomized PCA, the principal components are only approximated, so slightly more information will be discarded since the principal components won't point perfectly in the direction of highest variance.

The book describes reconstructions with RBF kernel PCA. This is more complex because the RBF kernel implicitly represents an infinite-dimensional feature space, making it not possible to invert the transformation from feature-space to the data-space (even though the linear projection from feature-space to reduced space is invertible) so reconstructions must be learned with supervised regression by predicting the preimage of a mapping to feature space that results in the minimum reconstruction error.

Reconstructions are apparently possible from Locally Linear Embeddings.

t-SNE cannot produce a reconstruction.

4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?

Yes, but only in certain cases. This proceedure can actually be fairly mindless because you can simply look at the singular values and search for the values past the "elbow" in the graph. However, in some cases, all of the dimensions might contain high variance, so the cost of discarding that dimension is high. This would correspond to a somewhat straight-lined singular value graph. From that perspective, the cutoff point of which dimensions to keep is not obvious.

5. Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?

That is not possible to say in advance. As described in my answer to question #4, the distribution of singular values might be fairly uniform, or more exponential in shape. For this reason it isn't possible to say how many dimensions will be needed to keep 95% of the explained variance. If the singular values are entirely uniform, then it would be necessary to keep 95% of the dimensions (950).

6. In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA, or Kernel PCA?

PCA: Works best for more linear datasets with lower dimensionality. This is due to the $O(m n^{2}) + O(n^{3})$ time complexity.

Randomized PCA: By default, Scikit-Learn switches between PCA and Randomized PCA based on the whether m-instances or n-features is > 500 and d is less than 80% of m or n. So Randomized PCA is faster than PCA when the goal is to substantially reduce the dimensionality of the dataset. These guidelines are probably good rules of thumb for manually choosing between PCA and randomized PCA.

Incremental PCA: If the dataset is too large for memory, incremental PCA makes it possible to perform PCA on mini-batches. This can be done in chunks via IncrementalPCA using explicit partial_fit, or by mmapping disk to memory and calling the usual fit method.

Kernel PCA: Kernel PCA is most useful when the data is highly non-linear, because it enables non-linear projections that standard PCA can't do.

7. How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?

Reconstruction error. This method chooses a metric that measures the distance between the original dataset and the reconstructed dataset after some form of prejection and inversion. An example metric is mean squared error.

It is also possible to measure the performance of a downstream supervised learning task and use this information to select a dimensionality reduction algorithm and tune its hyperparameters.

8. Does it make any sense to chain two different dimensionality reduction algorithms?

Speculatively, it might. One example might be to choose a more efficient algorithm to do the initial dimensionality reduction if the dimensionality or number of instances is very large, so the data does not fit in to memory. In this case, Incremental PCA could be chained with a more costly algorithm that does not offer an incremental variant.